In [ ]:
# import all libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split, StratifiedKFold
import requests, zipfile, io, os, math, codecs


In [ ]:
'''zip_file_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()'''

'zip_file_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"\nr = requests.get(zip_file_url)\nz = zipfile.ZipFile(io.BytesIO(r.content))\nz.extractall()'

In [ ]:
train_path='/content/drive/My Drive/sentiment/Train.csv'
test_path='/content/drive/My Drive/sentiment/Test.csv'
sub_path='/content/drive/My Drive/sentiment/SampleSubmission.csv'

train_df=pd.read_csv(train_path)
test_df=pd.read_csv(test_path)
sub_df=pd.read_csv(sub_path)

train_df.head()

,ID,text,label
0,13P0QT0,3sbaaaaaaaaaaaaaaaaaaaa lek ou le seim riahi o...,-1
1,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1
2,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1
3,U0TTYY8,ak slouma,1
4,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1


In [ ]:
full_data= pd.concat([train_df,test_df], ignore_index=True)

In [ ]:
from tqdm import tqdm
print('loading word embeddings...')

embeddings_index = {}
f = codecs.open('/content/wiki-news-300d-1M.vec', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))

1114it [00:00, 11135.34it/s]

loading word embeddings...


302643it [00:28, 10914.71it/s]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(train_df['text'],train_df['label'],stratify=train_df['label'], test_size=0.2, random_state=14)


In [ ]:
#Padding Function
s_len=25
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, truncating='post',padding = 'post')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(full_data['text'])
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

word_index = tokenizer.word_index


X_train = pad(X_train, length=s_len)
X_test = pad(X_test,length=s_len)


NameError: ignored

In [ ]:
#embedding matrix
embed_dim=300
print('preparing embedding matrix...')

words_not_found = []
embedding_matrix = np.zeros((len(word_index)+1, embed_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [ ]:
y_train_enc=pd.get_dummies(y_train)
y_test_enc=pd.get_dummies(y_test)

In [ ]:

model = Sequential()

model.add(Embedding(len(word_index)+1,embed_dim,input_length=s_len, weights=[embedding_matrix],trainable=True))
model.add(Bidirectional(LSTM(32,dropout=0.2, recurrent_dropout=0.2,return_sequences= True)))
model.add(Bidirectional(LSTM(32,dropout=0.3, recurrent_dropout=0.3,return_sequences= True)))
model.add(Bidirectional(LSTM(32,dropout=0.3, recurrent_dropout=0.3,return_sequences= True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(3,activation='softmax'))

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])
model.fit(X_train, y_train_enc, validation_data=(X_test, y_test_enc), epochs=6, batch_size=5000)

In [ ]:
model.evaluate(X_test, y_test_enc)

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_df.text)
test_seq = pad(test_seq,length=s_len)
preds=model.predict(test_seq)


In [ ]:
preds=np.argmax(preds, axis=1)
corr_pred=[]
for i in preds:
  if i==0:
    corr_pred.append(-1)
  elif i==1:
    corr_pred.append(0)
  else:
    corr_pred.append(1)

In [ ]:
sub=pd.DataFrame({'ID':test_df.ID.values, 'label':corr_pred})
sub.to_csv('fasttext.csv', index=False)

In [ ]:
 sub.label.value_counts()